In [1]:
import os
import re
import json
import numpy as np
from dotenv import load_dotenv
from mistralai import Mistral
import faiss


In [8]:
# 0. Charger la clé depuis le fichier .env
load_dotenv()
api_key = os.getenv("MISTRAL_API_KEY")
client = Mistral(api_key=api_key)

In [3]:
# 1. Lire fichiers .py et .ipynb
def read_code(folder, extensions={".py", ".ipynb"}, show_structure=False):
    """
    Parcourt un dossier récursivement et extrait le contenu des fichiers .py et .ipynb.
    
    Args:
        folder (str): chemin du dossier
        extensions (set): extensions de fichiers à inclure
        show_structure (bool): si True, affiche l’arborescence du dossier

    Returns:
        List[Tuple[str, str]]: liste de tuples (chemin_relatif, code_source)
    """
    code_files = []

    for dirpath, _, filenames in os.walk(folder):
        for f in filenames:
            ext = os.path.splitext(f)[1].lower()
            if ext in extensions:
                full_path = os.path.join(dirpath, f)
                rel_path = os.path.relpath(full_path, folder)
                try:
                    if ext == ".ipynb":
                        with open(full_path, "r", encoding="utf-8") as file:
                            nb = json.load(file)
                            code = ""
                            for cell in nb.get("cells", []):
                                if cell.get("cell_type") == "code":
                                    code += "".join(cell.get("source", [])) + "\n"
                            code_files.append((rel_path, code))
                    else:
                        with open(full_path, "r", encoding="utf-8") as file:
                            code = file.read()
                            code_files.append((rel_path, code))
                except Exception as e:
                    print(f"Erreur lecture {rel_path} : {e}")

    if show_structure:
        print("Contenu du dossier :")
        for path, _ in code_files:
            print(" -", path)

    return code_files


In [4]:
# 2. Découper par fonction
def split_functions(code):
    pattern = r"(def [\w_]+\s*\(.*?\):(?:\n(?:\s+.+))*)"
    funcs = re.findall(pattern, code, re.DOTALL)
    return funcs if funcs else [code]

# 3. Créer embeddings
import numpy as np
import time
import random

def create_embeddings(chunks, batch_size=16, max_retries=5):
    """
    Crée des embeddings pour une liste de textes (chunks) en batch
    et gère les erreurs 429 avec retry exponentiel.
    
    Args:
        chunks (list of str): textes à encoder
        batch_size (int): nombre de chunks à envoyer par requête
        max_retries (int): nombre de tentatives en cas d'erreur 429
    Returns:
        embeddings (list of np.array): embeddings de chaque chunk
    """
    embeddings = []

    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i+batch_size]
        for attempt in range(max_retries):
            try:
                resp = client.embeddings.create(
                    model="codestral-embed",
                    inputs=batch
                )
                # Ajouter chaque embedding du batch
                for emb in resp.data:
                    embeddings.append(np.array(emb.embedding, dtype=np.float32))
                break  # sortir de la boucle retry si OK
            except Exception as e:
                # Vérifie si c'est une erreur 429
                if "429" in str(e):
                    wait = 2 ** attempt + random.random()
                    print(f"Erreur 429, tentative {attempt+1}/{max_retries}. Attente {wait:.2f}s")
                    time.sleep(wait)
                else:
                    raise e  # autre erreur → remonter
    return embeddings



In [13]:
# 4. Lire + chunker + embeddings
folder = "/Users/aya31/Desktop/M2 MIASHS/Calcul Paralle/creating-a-README-for-a-codebase/TESTS/TEST2"
all_code = read_code(folder, show_structure=True)

all_chunks = []
chunk_to_path = []

for path, code in all_code:
    chunks = split_functions(code)
    all_chunks.extend(chunks)
    chunk_to_path.extend([path] * len(chunks))  # pour traçabilité


# 4 bis. Créer les embeddings à partir des chunks
embeddings = create_embeddings(all_chunks, batch_size=8)


# 5. Stocker dans FAISS
dim = len(embeddings[0])
index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings))

# 6. Fonction pour poser une question au LLM avec RAG
def ask_question(query, top_k=3):
    """
    Pose une question sur le code en utilisant RAG avec Mistral plus léger.
    """
    # 1. Créer l'embedding de la question
    query_emb = client.embeddings.create(
        model="codestral-embed",  # embeddings restent les mêmes
        inputs=[query]
    )
    q_vec = np.array(query_emb.data[0].embedding, dtype=np.float32).reshape(1, -1)

    # 2. Chercher les chunks les plus proches dans FAISS
    D, I = index.search(q_vec, k=top_k)
    context = "\n\n".join([all_chunks[i] for i in I[0]])

    # 3. Préparer le prompt pour le LLM
    prompt = (
        "Voici le code extrait :\n"
        f"{context}\n\n"
        f"Question : {query}\n"
        "Réponds de manière claire et précise, explique le fonctionnement du code si nécessaire. Donne les bibliothèques utilisées et à installer. "
    )

    # 4. Appeler le LLM Mistral plus léger
    response = client.chat.complete(
        model="codestral-2508",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        max_tokens=600
    )

    return response.choices[0].message.content


# 7. Exemple
question = "Redige un readme (gitub) pour ce dossier"
answer = ask_question(question)
print("Réponse du LLM :\n", answer)

Contenu du dossier :
 - code3.py
 - DOSSIER 2/code2.py
 - DOSSIER 2/code4.ipynb
 - DOSSIER 1/code5.ipynb
 - DOSSIER 1/code1.py
Réponse du LLM :
 # README.md

## Description

Ce dépôt contient un ensemble de fonctions utilitaires pour la manipulation de fichiers, de calculs mathématiques et de création de DataFrames. Le code est organisé en plusieurs fonctions indépendantes pour faciliter son utilisation dans divers projets.

## Fonctions disponibles

### Fonctions de manipulation de fichiers
- `list_files(folder)` : Liste les fichiers dans un dossier spécifié.
- `create_folder(folder)` : Crée un dossier s'il n'existe pas déjà.
- `create_file(path, content)` : Crée un fichier avec le contenu spécifié.
- `read_file(path)` : Lit le contenu d'un fichier.
- `count_lines(file_path)` : Compte le nombre de lignes dans un fichier.

### Fonctions mathématiques
- `add(a, b)` : Additionne deux nombres.
- `multiply(a, b)` : Multiplie deux nombres.
- `divide(a, b)` : Divise deux nombres (avec gestio

In [ ]:
print(client.models.list())



object='list' data=[BaseModelCard(id='mistral-medium-2505', capabilities=ModelCapabilities(completion_chat=True, completion_fim=False, function_calling=True, fine_tuning=True, vision=True, classification=False), object='model', created=1763399316, owned_by='mistralai', name='mistral-medium-2505', description='Our frontier-class multimodal model released May 2025.', max_context_length=131072, aliases=[], deprecation=None, deprecation_replacement_model=None, default_model_temperature=0.3, TYPE='base'), BaseModelCard(id='mistral-large-latest', capabilities=ModelCapabilities(completion_chat=True, completion_fim=False, function_calling=True, fine_tuning=True, vision=True, classification=False), object='model', created=1763399316, owned_by='mistralai', name='mistral-large-latest', description='Official mistral-large-latest Mistral AI model', max_context_length=131072, aliases=[], deprecation=None, deprecation_replacement_model=None, default_model_temperature=0.3, TYPE='base'), BaseModelCard(